In [1]:
import pandas as pd
import tensorflow as tf
import re
import nltk
import preprocessor as p
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

2023-04-13 19:18:02.709904: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-13 19:18:03.148045: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-13 19:18:03.148070: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-13 19:18:04.316832: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
df = pd.read_csv('./datasets/short.csv')
df.head()

,Unnamed: 0,Text,Language,Label
0,0,@Charlie_Corley @Kristine1G @amyklobuchar @Sty...,en,litigious
1,2,https://t.co/YJNiO0p1JV Flagstar Bank disclose...,en,litigious
2,3,Rwanda is set to host the headquarters of Unit...,en,positive
3,4,OOPS. I typed her name incorrectly (today’s br...,en,litigious
4,5,It sucks for me since I'm focused on the natur...,en,negative


In [3]:
df = df.sample(frac=0.1)

len(df)

18594

In [4]:
df.groupby('Label').size()

Label
litigious      3900
negative       5243
positive       5286
uncertainty    4165
dtype: int64

In [5]:
def clean_text(text):    
    text = re.sub(r"http[^ ]* ", '', text)
    text = re.sub(r"#\S* ", '', text)
    text = re.sub(r"@\S* ", '', text)
    text = re.sub(r"[^\w\s]", '', text)
    text = re.sub(r"\s\s+", ' ', text)
    text = text.lower()
    return text

In [6]:
texts = [clean_text(x) for x in df['Text']]
labels = []
for s in df['Label']:
    arr = np.zeros(4)
    i = 0
    if s == 'negative':
        i = 1
    elif s == 'litigious':
        i = 2
    elif s == 'uncertainty':
        i = 3
    arr[i] = 1.0
    labels.append(arr)
        

In [7]:
labels[0]

array([0., 0., 1., 0.])

In [8]:
vocab_size = 20000
embedding_dim = 32
max_length = 300
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

training_size = int(len(texts) * 0.70)
training_sentences = texts[0:training_size]
testing_sentences = texts[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

In [9]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [10]:
len(word_index)

30025

In [11]:
print(type(training_padded), training_padded[0].shape)

<class 'numpy.ndarray'> (300,)


In [12]:
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [13]:
# model = tf.keras.Sequential([
#     tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dense(4, activation='sigmoid')
# ])
# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
# model.summary()

model = tf.keras.models.load_model('./saved_models/classifier2LaksData3rd.h5',compile = False)


2023-04-13 19:18:09.255649: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-13 19:18:09.255796: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-04-13 19:18:09.255880: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-04-13 19:18:09.255977: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2023-04-13 19:18:09.256056: W tensorfl

In [14]:
# num_epochs = 8
# history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=1)

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [15]:
vals = ["Positive", "Negative", "Litigious", "Uncertain", "Neutral"]
def prediction(s):
    sequences = tokenizer.texts_to_sequences([s])
    padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
    print(type(padded))
    x = model.predict(padded)[0]
    max_index = 0
    for i in range(0,4):
        if x[i] > x[max_index]:
            max_index = i
    
    if x[max_index] < 0.3:
        max_index = 4
    print(f"The sentence is {vals[max_index]}")
    
    return x
    # if x > 0.7:
    #     return f"Positive: confidence {x}" 
    # elif x < 0.33:
    #     return f"Negative: confidence {x}"
    # return f"Can't say: confidence {x}"

In [22]:
prediction("i am not in a good mood")

<class 'numpy.ndarray'>
1/1 [==============================] - 0s 31ms/step
The sentence is Positive


array([9.98654664e-01, 2.58125510e-04, 5.16637287e-04, 1.31805455e-05],
      dtype=float32)

In [ ]:
# model.save('./saved_models/classifier2LaksData_1.h5')

In [ ]:
import tensorflowjs as tfjs


In [ ]:
# tfjs.converters.save_keras_model(model, './frontend/models')

In [ ]:
# import json

# save_file = open("savedata.json", "w")  
# json.dump(word_index, save_file, indent = 4)  
# save_file.close()  